# Data Extraction


In [6]:
!pip install pymysql
!pip install sqlalchemy

In [7]:
# Imorting all necessary libraries
import pandas as pd
import pymysql
import sqlalchemy
from sqlalchemy import create_engine

**Creating Connection between SQL & Python**

In [8]:
# MySQL credentials
username = "root"               # MySQL username
password = "8898"               # MySQL password
host = "localhost"              # Her my MySQL sever running on localhost
port = 3306                     # MySQL default port
database = "Customers_Project"  # Database name

In [9]:
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [10]:
# Test connection - show all tables
try:
    df = pd.read_sql("SHOW TABLES;", con=engine)
    print("✅ Connection successful. Tables in DB:")
    print(df)
except Exception as e:
    print("❌ Connection failed:")
    print(e)

✅ Connection successful. Tables in DB:
  Tables_in_customers_project
0           customer_purchase
1                   customers
2                    products
3                transactions


**Python script to extract data from the SQL database**

In [11]:
# Loading Pandas DataFrame from SQL Tables
customers = pd.read_sql_table("customers", engine)
products = pd.read_sql_table("products", engine)
transactions = pd.read_sql_table("transactions", engine)

print(customers.head())
print(products.head())
print(transactions.head())

   CustomerKey  CustomerID        CustomerName                      Country
0            1         887    Kenneth Martinez                     Barbados
1            2         560     Joseph Anderson     Northern Mariana Islands
2            3         701    Vincent Reynolds       British Virgin Islands
3            4         630  Christopher Morris                    Guatemala
4            5         631          Sarah King  Falkland Islands (Malvinas)
   ProductKey  ProductID      ProductName  ProductCategory
0           1        240           Router      Electronics
1           2        299           Camera      Electronics
2           3        207  Electric Kettle  Home Appliances
3           4        290       Smartwatch      Electronics
4           5        281          Toaster  Home Appliances
   TransactionID  CustomerKey  ProductKey  PurchaseQuantity  PurchasePrice  \
0              1            1           1                 5         689.99   
1              2            2     

**Merging Tables in a Dataframe**

In [12]:
# Merge Transactions with Customers
merged_df = pd.merge(transactions, customers, on ='CustomerKey', how='left')

# Transactions + Products join (ProductKey ke through)
merged_df = pd.merge(merged_df, products, on="ProductKey", how="left")

#  Cheking Output
print(merged_df.head())

   TransactionID  CustomerKey  ProductKey  PurchaseQuantity  PurchasePrice  \
0              1            1           1                 5         689.99   
1              2            2           2                 4          79.27   
2              3            3           3                 3         666.75   
3              4            4           4                 5         316.19   
4              5            5           5                 4         700.24   

  PurchaseDate  CustomerID        CustomerName                      Country  \
0   2024-03-01         887    Kenneth Martinez                     Barbados   
1   2024-01-26         560     Joseph Anderson     Northern Mariana Islands   
2   2024-05-13         701    Vincent Reynolds       British Virgin Islands   
3   2023-09-21         630  Christopher Morris                    Guatemala   
4   2024-01-25         631          Sarah King  Falkland Islands (Malvinas)   

   ProductID      ProductName  ProductCategory  
0      

# Data Analysis
**1. Calculate total purchases, total revenue, and average purchase value.**

In [13]:
# 1. Total Purchases (sum of quantity column)
total_purchases = merged_df['PurchaseQuantity'].sum()
print("Total Purchases:", total_purchases)

# 2. Total Revenue (Quantity * Price, then sum)
merged_df['Revenue'] = merged_df['PurchaseQuantity'] * merged_df['PurchasePrice']
total_revenue = merged_df['Revenue'].sum()
print("Total Revenue:", total_revenue)

# 3. Average Purchase Value (Total Revenue / Total Purchases)
average_purchase_value = total_revenue / total_purchases
print("Average Purchase Value:", average_purchase_value)

Total Purchases: 3042
Total Revenue: 1481506.85
Average Purchase Value: 487.01737343852733


**2. Identify top customers and their purchasing behavior.**

In [14]:
# Group data by customer and calculate Total revenue & Total purchases
customer_stats = (
    merged_df.groupby('CustomerKey')
    .agg(
        TotalRevenue=('Revenue', 'sum'),
        TotalPurchases=('PurchaseQuantity', 'sum')
    )
    .sort_values(by='TotalRevenue', ascending=False)
)

print("Top 5 Customers by Revenue:")
print(customer_stats.head(5))

Top 5 Customers by Revenue:
             TotalRevenue  TotalPurchases
CustomerKey                              
794               4997.20               5
316               4990.85               5
455               4967.75               5
485               4952.10               5
86                4950.65               5


In [16]:
# Avg Purchase Value for checking spending pattern
customer_stats['AvgPurchaseValue'] = (
    customer_stats['TotalRevenue'] / customer_stats['TotalPurchases']
)
customer_stats = customer_stats.sort_values(by='AvgPurchaseValue', ascending=False)

print("Customer with Avg Purchase Value:")
print(customer_stats.head())

Customer with Avg Purchase Value:
             TotalRevenue  TotalPurchases  AvgPurchaseValue
CustomerKey                                                
266               1999.96               2            999.98
794               4997.20               5            999.44
316               4990.85               5            998.17
137               2992.35               3            997.45
363               3984.04               4            996.01


**3. Analyze purchase trends over time (monthly, quarterly, yearly).**

In [17]:
# Make sure PurchaseDate is datetime
print(merged_df['PurchaseDate'].dtype)

datetime64[ns]


In [18]:
# Extract Year, Month, Quarter
merged_df['Year'] = merged_df['PurchaseDate'].dt.year
merged_df['Month'] = merged_df['PurchaseDate'].dt.month
merged_df['Quarter'] = merged_df['PurchaseDate'].dt.to_period('Q')

In [19]:
# Yearly Trends
yearly_trends = (
    merged_df.groupby('Year')
    .agg(TotalQuantity=('PurchaseQuantity', 'sum'), TotalRevenue=('Revenue', 'sum')).reset_index()
)

print("Yearly Trends:\n", yearly_trends)

Yearly Trends:
    Year  TotalQuantity  TotalRevenue
0  2023           1647     790514.96
1  2024           1395     690991.89


In [20]:
# Monthly Trends
monthly_trends = (
    merged_df.groupby(['Year', 'Month'])
    .agg(TotalQuantity=('PurchaseQuantity', 'sum'), TotalRevenue=('Revenue', 'sum')).reset_index()
)

print("Monthly Trends:\n", monthly_trends)

Monthly Trends:
     Year  Month  TotalQuantity  TotalRevenue
0   2023      6             27       8258.62
1   2023      7            269     140528.32
2   2023      8            233      95945.72
3   2023      9            287     143595.40
4   2023     10            243     124279.18
5   2023     11            293     122373.42
6   2023     12            295     155534.30
7   2024      1            290     134016.35
8   2024      2            272     128590.54
9   2024      3            203     111947.28
10  2024      4            195     102830.12
11  2024      5            268     133782.95
12  2024      6            167      79824.65


In [21]:
# Quarterly Trends
quarterly_trends = (
    merged_df.groupby('Quarter')
    .agg(TotalQuantity=('PurchaseQuantity', 'sum'), TotalRevenue=('Revenue', 'sum')).reset_index()
)

print("Quarterly Trends:\n", quarterly_trends)

Quarterly Trends:
   Quarter  TotalQuantity  TotalRevenue
0  2023Q2             27       8258.62
1  2023Q3            789     380069.44
2  2023Q4            831     402186.90
3  2024Q1            765     374554.17
4  2024Q2            630     316437.72


**4. Identify the top-performing product categories.**

In [22]:
# Group by ProductCategory and calculate total revenue & quantity
top_categories = merged_df.groupby('ProductCategory').agg(
    TotalRevenue=('Revenue', 'sum'),
    TotalPurchases=('PurchaseQuantity', 'sum')
).sort_values(by='TotalRevenue', ascending=False).reset_index()

print("\nTop Performing Product Categories:\n", top_categories)


Top Performing Product Categories:
    ProductCategory  TotalRevenue  TotalPurchases
0  Home Appliances     745538.64            1563
1      Electronics     735968.21            1479


**5. Generate a summary report with key insights.**

# Customer Purchase Behavior Summary Report
**Total Purchases: 3,042       
Total Revenue: Rs. 1,481,506.85         
Average Purchase Value: ₹487.02**      

**Top 5 Customers by Revenue:**      
Customers 794, 316, 455, 485, 86 — each with around ₹4,950 to ₹5,000 revenue and 5 purchases each.      
**Customers with Highest Average Purchase Value:**     
Customers 266, 794, 316, 137, 363 — with average purchase values close to ₹1,000.   

**Yearly Sales Trends:**
* 2023: 1,647 units sold, revenue ₹790,514.96

* 2024: 1,395 units sold, revenue ₹690,991.89

**Quarterly Sales Trends:**      
* Highest revenue in Q3 and Q4 2023 (~₹380,000 and ₹402,000 respectively)

* Moderate sales continuing in early 2024

**Monthly Sales Trends (Sample):**
* June 2023: ₹8,258.62 from 27 purchases

* July 2023: ₹140,528.32 from 269 purchases

* August 2023: ₹95,945.72 from 233 purchases

**Top Performing Product Categories:**
* Home Appliances: ₹745,538.64 revenue from 1,563 purchases

* Electronics: ₹735,968.21 revenue from 1,479 purchases

**Insights:**
* The majority of revenue comes from Home Appliances and Electronics.

* Top customers are consistent in purchases and contribute significantly to revenue.

* Q3 and Q4 are peak sales quarters, possibly due to seasonal factors.

* Focus marketing efforts on these product categories and peak quarters for maximum impact.

